In [1]:
!pip install -q ultralytics thop --no-deps

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.4 MB/s eta 0:00:00


# SETUP & DATA CHECK

In [2]:
import os
import glob
import random
import time
import yaml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
from ultralytics import YOLO
from ultralytics.nn.tasks import DetectionModel

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [3]:
def check_dataset_integrity(data_root_path, splits=["train", "test"]):
    """
    Kiểm tra số lượng ảnh và nhãn trong folder dataset.
    Args:
        data_root_path (str): Đường dẫn gốc tới folder dataset
        splits (list): Các tập con cần check (train, test)
    """
    print(f"\n🔍 CHECKING DATASET AT: {data_root_path}")
    
    for split in splits:
        img_dir = os.path.join(data_root_path, "images", split)
        lbl_dir = os.path.join(data_root_path, "labels", split)
        
        # Check path exists
        if not os.path.exists(img_dir):
            print(f"❌ Không tìm thấy thư mục: {img_dir}")
            continue

        imgs = sorted(glob.glob(f"{img_dir}/*.jpg"))
        lbls = sorted(glob.glob(f"{lbl_dir}/*.txt"))
        
        img_stems = {os.path.splitext(os.path.basename(p))[0] for p in imgs}
        lbl_stems = {os.path.splitext(os.path.basename(p))[0] for p in lbls}

        no_label = img_stems - lbl_stems
        orphan_lbl = lbl_stems - img_stems

        print(f"[{split.upper()}]")
        print(f"   - Ảnh: {len(imgs)} | Nhãn: {len(lbls)}")
        print(f"   - Ảnh thiếu nhãn: {len(no_label)}")
        print(f"   - Nhãn mồ côi (không có ảnh): {len(orphan_lbl)}")
    print("-" * 50)

In [4]:
DATASET_TT100K = "/kaggle/input/build-dataset-tt100k-tsd/tt100k_yolo"
DATASET_VNTSR = "/kaggle/input/build-dataset-vn-tsd/vntsr_yolo"

check_dataset_integrity(DATASET_TT100K)
check_dataset_integrity(DATASET_VNTSR)


🔍 CHECKING DATASET AT: /kaggle/input/build-dataset-tt100k-tsd/tt100k_yolo
[TRAIN]
   - Ảnh: 6034 | Nhãn: 6034
   - Ảnh thiếu nhãn: 0
   - Nhãn mồ côi (không có ảnh): 0
[TEST]
   - Ảnh: 3022 | Nhãn: 3022
   - Ảnh thiếu nhãn: 0
   - Nhãn mồ côi (không có ảnh): 0
--------------------------------------------------

🔍 CHECKING DATASET AT: /kaggle/input/build-dataset-vn-tsd/vntsr_yolo
[TRAIN]
   - Ảnh: 2552 | Nhãn: 2552
   - Ảnh thiếu nhãn: 0
   - Nhãn mồ côi (không có ảnh): 0
[TEST]
   - Ảnh: 639 | Nhãn: 639
   - Ảnh thiếu nhãn: 0
   - Nhãn mồ côi (không có ảnh): 0
--------------------------------------------------


# MODEL CONFIG GENERATOR

In [5]:
def create_yolo_config(width_scale, depth_scale, save_name, save_dir="/kaggle/working/configs"):
    """
    Tạo file YAML cấu hình cho YOLOv8n custom với width/depth tùy chỉnh.
    Args:
        width_scale (float): Tỷ lệ chiều rộng 
        depth_scale (float): Tỷ lệ độ sâu
        save_name (str): Tên file lưu (vd: yolo_lite_pi_v1.yaml)
    """
    os.makedirs(save_dir, exist_ok=True)
    
    # Base channels của YOLOv8n gốc
    base_channels = [64, 128, 256, 512, 1024] # P1, P2, P3, P4, P5
    
    # Tính toán channels dựa trên width_scale
    # c1=P1/2, c2=P2/4, c3=P3/8, c4=P4/16, c5=P5/32
    c = [int(x * width_scale) for x in base_channels]
    
    # Cấu trúc mạng 
    model_cfg = {
        "nc": 1,             
        "reg_max": 8,       
        "depth_multiple": depth_scale, # Scale depth
        "width_multiple": width_scale, # Scale width (để reference)
        
        # Backbone
        "backbone": [
            [-1, 1, "Conv", [c[0], 3, 2]],          # 0-P1/2 
            [-1, 1, "Conv", [c[1], 3, 2]],          # 1-P2/4 
            [-1, 3, "C2f",  [c[1], True]],          # 2
            [-1, 1, "Conv", [c[2], 3, 2]],          # 3-P3/8 
            [-1, 6, "C2f",  [c[2], True]],          # 4
            [-1, 1, "Conv", [c[3], 3, 2]],          # 5-P4/16 
            [-1, 6, "C2f",  [c[3], True]],          # 6
            [-1, 1, "Conv", [c[4], 3, 2]],          # 7-P5/32 
            [-1, 3, "C2f",  [c[4], True]],          # 8
            [-1, 1, "SPPF", [c[4], 5]],             # 9
        ],
        
        # Head
        "head": [
            [-1, 1, "nn.Upsample", [None, 2, "nearest"]],
            [[-1, 6], 1, "Concat", [1]],            # concat P4
            [-1, 3, "C2f", [c[3], True]],           # 12 

            [-1, 1, "nn.Upsample", [None, 2, "nearest"]],
            [[-1, 4], 1, "Concat", [1]],            # concat P3
            [-1, 3, "C2f", [c[2], True]],           # 15 (P3 head)

            [-1, 1, "Conv", [c[2], 3, 2]],          # 16 ↓ x2
            [[-1, 12], 1, "Concat", [1]],           # concat P4
            [-1, 3, "C2f", [c[3], True]],           # 18 (P4 head)

            [-1, 1, "Conv", [c[3], 3, 2]],          # 19 ↓ x2
            [[-1, 9], 1, "Concat", [1]],            # concat P5
            [-1, 3, "C2f", [c[4], True]],           # 21 (P5 head)

            [[15, 18, 21], 1, "Detect", [1]],       # 22 Detect
        ],
    }
    
    save_path = os.path.join(save_dir, save_name)
    with open(save_path, "w") as f:
        yaml.dump(model_cfg, f, sort_keys=False)
    
    print(f"✅ Created Config: {save_name} | Width: {width_scale} | Channels: {c}")
    return save_path

# DATA YAML SETUP

In [6]:
def create_data_yaml(data_root_path, save_path="/kaggle/working/data.yaml"):
    """
    Tạo file data.yaml trỏ tới dataset.
    """
    yaml_dict = {
        "path": data_root_path,
        "train": os.path.join(data_root_path, "images/train"),
        "val":   os.path.join(data_root_path, "images/test"), # Lưu ý: TT100K của bạn dùng folder 'test' làm val
        "test":  os.path.join(data_root_path, "images/test"),
        "nc": 1,
        "names": ["sign"]
    }
    
    with open(save_path, "w") as f:
        yaml.dump(yaml_dict, f, sort_keys=False)
    
    print(f"✅ Created Data YAML at: {save_path}")
    print(f"   Train path: {yaml_dict['train']}")
    return save_path

# TRAINING FUNCTION

In [7]:
def train_ablation_variant(model_name, cfg_yaml, data_yaml, project_dir, 
                           epochs=50, batch=16, imgsz=640, device="0"):
    """
    Train và đo lường hiệu năng (FPS, Params, GFLOPs) của biến thể model.
    """
    print(f"\n🚀 START TRAINING: {model_name}")
    print(f"   Config: {cfg_yaml}")
    
    # 1. Load model từ config YAML (Build from scratch)
    model = YOLO(cfg_yaml, task="detect")
    
    # Lấy thông tin Params và GFLOPs trước khi train
    # model.info() trả về tuple (layers, params, gradients, flops)
    try:
        info = model.info(verbose=False)
        params_m = round(info[1] / 1e6, 2) # Triệu tham số
        flops_g  = round(info[3], 2)       # GFLOPs
    except:
        params_m = 0
        flops_g = 0
    
    # 2. Train
    start_time = time.time()
    results = model.train(
        data=data_yaml,
        imgsz=imgsz,
        epochs=epochs,
        batch=batch,
        project=project_dir,
        name=model_name,
        device=device,
        mosaic=0.7,      
        scale=0.5,
        copy_paste=0.05,
        hsv_h=0.015, hsv_s=0.7, hsv_v=0.4,
        exist_ok=True,
        verbose=False,   
        plots=False      
    )
    
    train_time = round((time.time() - start_time) / 60, 2)
    
    # 3. Validation & Benchmark Speed
    # Chúng ta load lại weights tốt nhất để đo cho chuẩn
    best_weight_path = os.path.join(project_dir, model_name, "weights", "best.pt")
    val_model = YOLO(best_weight_path)
    
    # Chạy validation (lấy mAP và tốc độ)
    print(f"   ...Benchmarking speed on validation set...")
    val_metrics = val_model.val(data=data_yaml, split='test', verbose=False)
    
    # Lấy mAP
    map50 = round(val_metrics.results_dict.get("metrics/mAP50(B)", 0), 4)
    map5095 = round(val_metrics.results_dict.get("metrics/mAP50-95(B)", 0), 4)
    
    # Lấy tốc độ (Ultralytics trả về dict 'speed' đơn vị ms/img)
    # speed = {'preprocess': x, 'inference': y, 'postprocess': z}
    speed_ms = val_metrics.speed['inference']
    fps = round(1000.0 / speed_ms, 2) # FPS = 1000 / ms
    
    summary = {
        "Model": model_name,
        "Params(M)": params_m,     # Reviewer thích cái này
        "FLOPs(G)": flops_g,       # Reviewer thích cái này
        "mAP50": map50,
        "mAP50-95": map5095,
        "Inference(ms)": round(speed_ms, 2),
        "FPS (GPU)": fps,          # Tốc độ trên Kaggle
        "TrainTime(min)": train_time,
    }
    
    print(f"FINISHED {model_name} | mAP50: {map50} | FPS: {fps} | Params: {params_m}M")
    return summary

# RUN ABLATION STUDY ON MULTIPLE DATASETS

In [8]:

DATASET_TT100K = "/kaggle/input/build-dataset-tt100k-tsd/tt100k_yolo"
DATASET_VNTSR  = "/kaggle/input/build-dataset-vn-tsd/vntsr_yolo"

def run_ablation_on_dataset(dataset_path, dataset_name, epochs=30):
    """
    Chạy thực nghiệm so sánh 3 biến thể trên một dataset cụ thể.
    Trả về danh sách kết quả.
    """
    print(f"\n{'='*20} START ABLATION FOR: {dataset_name} {'='*20}")
    
    # Lưu tên file khác nhau để tránh xung đột
    yaml_save_path = f"/kaggle/working/data_{dataset_name.lower()}.yaml"
    data_yaml_path = create_data_yaml(dataset_path, save_path=yaml_save_path)
    
    project_dir = f"/kaggle/working/Ablation_{dataset_name}"
    
    # C. Định nghĩa 3 biến thể
    variants = [
        # --- NHÓM 1: WIDTH ABLATION (Giữ Depth=0.33, Thay đổi Width) ---
        # Chứng minh: 0.75 là cân bằng, 0.5 quá yếu, 1.0 thừa thãi
        {"name": f"{dataset_name}_Var_Width_Small", "w": 0.50, "d": 0.33}, 
        {"name": f"{dataset_name}_YOLO_LitePi",     "w": 0.75, "d": 0.33}, # <-- OUR BEST MODEL
        {"name": f"{dataset_name}_Var_Width_Base",  "w": 1.00, "d": 0.33}, 
        
        # --- NHÓM 2: DEPTH ABLATION (Giữ Width=0.75, Thay đổi Depth) ---
        # Chứng minh: Depth=0.33 đủ dùng, Depth=0.67 (sâu hơn) chỉ làm chậm máy mà không tăng mAP
        # Lưu ý: Ta tái sử dụng model 'YOLO_LitePi' (0.75, 0.33) làm mốc so sánh
        {"name": f"{dataset_name}_Var_Depth_Deep",  "w": 0.75, "d": 0.67},
    ]
    
    dataset_results = []
    
    for v in variants:
        # Tạo config model
        cfg_name = f"{v['name']}.yaml"
        cfg_path = create_yolo_config(width_scale=v['w'], depth_scale=v['d'], save_name=cfg_name)
        
        # Run Training
        res = train_ablation_variant(
            model_name=v['name'],
            cfg_yaml=cfg_path,
            data_yaml=data_yaml_path,
            project_dir=project_dir,
            epochs=epochs,
            batch=16,
            imgsz=640
        )
        
        # Bổ sung thông tin định danh để sau này gộp bảng
        res["Dataset"] = dataset_name
        res["Width"] = v['w']
        res["Depth"] = v['d']
        
        dataset_results.append(res)
        
    return dataset_results

# =======================================================
# 🚀 MAIN EXECUTION
# =======================================================

final_results_list = []

# --- CHẠY DATASET 1: TT100K ---
results_tt100k = run_ablation_on_dataset(DATASET_TT100K, "TT100K", epochs=30)
final_results_list.extend(results_tt100k)

# --- CHẠY DATASET 2: VNTSR ---
# Lưu ý: VN-Signs có thể nhỏ hơn nên train nhanh hơn
results_vntsr = run_ablation_on_dataset(DATASET_VNTSR, "VNTSR", epochs=30) 
final_results_list.extend(results_vntsr)

# =======================================================
# 📊 TỔNG HỢP VÀ HIỂN THỊ KẾT QUẢ
# =======================================================
df_results = pd.DataFrame(final_results_list)

# Sắp xếp cột cho khoa học
cols_order = [
    "Dataset", "Model", 
    "Width", "Depth", "Params(M)", "FLOPs(G)", 
    "mAP50", "mAP50-95", "FPS (GPU)", "Inference(ms)"
]

cols_final = [c for c in cols_order if c in df_results.columns]
df_results = df_results[cols_final]

print("\n=== FINAL ABLATION STUDY RESULTS (MASTER TABLE) ===")
display(df_results)

# Lưu file CSV để tải về vẽ biểu đồ
csv_path = "/kaggle/working/ablation_study_full_results.csv"
df_results.to_csv(csv_path, index=False)
print(f"✅ Đã lưu bảng kết quả tổng hợp tại: {csv_path}")


==================== START ABLATION FOR: TT100K ====================
✅ Created Data YAML at: /kaggle/working/data_tt100k.yaml
   Train path: /kaggle/input/build-dataset-tt100k-tsd/tt100k_yolo/images/train
✅ Created Config: TT100K_Var_Width_Small.yaml | Width: 0.5 | Channels: [32, 64, 128, 256, 512]

🚀 START TRAINING: TT100K_Var_Width_Small
   Config: /kaggle/working/configs/TT100K_Var_Width_Small.yaml
Ultralytics 8.3.252 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla P100-PCIE-16GB, 16269MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.05, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data_tt100k.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fract

,Dataset,Model,Width,Depth,Params(M),FLOPs(G),mAP50,mAP50-95,FPS (GPU),Inference(ms)
0,TT100K,TT100K_Var_Width_Small,0.50,0.33,0,0,0.8922,0.6336,461.42,2.17
1,TT100K,TT100K_YOLO_LitePi,0.75,0.33,0,0,0.9253,0.6814,153.26,6.52
2,TT100K,TT100K_Var_Width_Base,1.00,0.33,0,0,0.9358,0.6968,74.11,13.49
3,TT100K,TT100K_Var_Depth_Deep,0.75,0.67,0,0,0.9152,0.6729,118.82,8.42
4,VNTSR,VNTSR_Var_Width_Small,0.50,0.33,0,0,0.9541,0.7051,597.37,1.67
5,VNTSR,VNTSR_YOLO_LitePi,0.75,0.33,0,0,0.9730,0.7416,250.83,3.99
6,VNTSR,VNTSR_Var_Width_Base,1.00,0.33,0,0,0.9812,0.7516,123.02,8.13
7,VNTSR,VNTSR_Var_Depth_Deep,0.75,0.67,0,0,0.9772,0.7434,200.13,5.00


✅ Đã lưu bảng kết quả tổng hợp tại: /kaggle/working/ablation_study_full_results.csv
